<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/clean-empty-space/15_optimization/010_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 최적화 Optimization



## 수식 표시



The minimum value of function $f(x)$ given that $x$ is an element of a set $S$ is $y_{minimum}$<br>집합 $S$의 원소 $x$ 가운데 함수 $f(x)$의 최소값 $y_{minimum}$


$$
y_{minimum} = \underset{x \in S}{\min} f(x)
$$



Among the elements of a set $S$, $x_{minimizing}$ would minimize the function $f(x)$.
<br>집합 $S$의 원소 $x$ 가운데 함수 $f(x)$을 최소로 만드는 $x$값 $x_{minimizing}$


$$
x_{minimizing} = \underset{x \in S}{\arg \min} f(x)
$$



Convexity<br>
함수 $f(x)$ 는 $- \infty $ 방향에서 $+ \infty $ 방향을 바라볼 때 **볼록**한가?



$$
f(tx + (1-t)y) \le tf(x) + (1-t)f(y)
$$



![Convexity](https://upload.wikimedia.org/wikipedia/commons/c/c2/Convex-function-graph-1.png)


Is the function $f(\cdot)$ **always below** a straight line between two points on it $(x, f(x))$ and $(y, f(y))$?
<br>함수 $ f(\cdot) $ 위의 두 점 $\left(x,f(x)\right)$ 과 $\left(t,f(y)\right)$ 을 직선으로 이었을 때, $f(\cdot)$ 은 **항상 그 아래**를 지나는가?



## Quick Example<br>간단한 사례



Import `scipy`'s sub-package `optimize`<br>
`scipy` 패키지의 `optimize` 부 패키지를 읽어들임<br>[Scipy Documentation](https://docs.scipy.org/doc/scipy/reference/optimize.html)



In [ ]:
import scipy.optimize as so



최적화할 `cost_function` 을 선언함<br>
Declare `cost_function` to optimize



In [ ]:
def cost_function(x:float, x_min:float=10.0):
    return (x-x_min) * (x-x_min)



위 함수를 매개변수로 `minimize_scalar()`를 호출<br>
Call `minimize_scalar()` using the function above as an argument



In [ ]:
%%time
result = so.minimize_scalar(cost_function)



In [ ]:
result



(Optional) To observe how `minimize_scalar()` found `result`, *hook* the cost function to plot intermediate steps<br>
(선택) `minimize_scalar()` 가 `result` 를 어떻게 찾는지 중간 과정을 알고싶다면, 비용 함수를 *가로채* 단계별 그래프를 그림



In [ ]:
import matplotlib.pyplot as plt
import numpy as np


original_cost_function = cost_function

def my_new_cost_function_with_plot(x):
    x_plot = np.linspace(-25, 25)
    y_plot = original_cost_function(x_plot)
    plt.plot(x_plot, y_plot)
    result = original_cost_function(x)
    plt.plot(x, result, 'o')
    plt.title(f"x = {x}")
    plt.grid(True)
    plt.show()
    plt.close()
    return result

cost_function = my_new_cost_function_with_plot



In the Google colab, immediately after running the following cell, before the plots appear, select the cell below.<br>
구글 코랩에서는, 아래 셀을 실행시킨 직후, 그림이 나타나기 전, 그 아래 셀을 선택하시오.



In [ ]:
%%time
# to save CI time
import os
if os.getenv('CI', False):
    options = {'maxiter': 1}
else:
    options = None

result = so.minimize_scalar(cost_function, options=options)



In [ ]:
result



## 도움말<br>Help



Help on function fmin in module scipy.optimize.optimize:<br>
`scipy.optimize.optimize` 모듈의 `minimize()` 함수에 관한 도움말:<br>
[link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)



In [ ]:
help(so.minimize)



### Parameters<br>매개변수



#### `func` : callable `func(x,*args)`



First argument is the name of the function to minimize. It may have following form:<br>
첫번째 매개변수는 최적화할 함수의 이름. 예를 들어 함수의 형태는 다음과 같음:



In [ ]:
def second_order_polynomial(x, *args):
    a, b, c = args
    cost = a * x * x + b * x + c
    return cost



어떤 함수가 매개변수로 다른 함수를 받아들여서 호출하는 경우, 호출된 함수는 **callback function**<br>
If a function takes another function as an argument and calls it, the called function is a **callback function**




`args` above allows us optionally pass more arguments to the cost function in addition to the `x`.<br>
위 `args`를 이용하여 비용함수에 추가적으로 다른 매개변수를 전달할 수 있음.



If we want to calculate a second order polynomial $x ^ 2 - 2x - 1$ when $x=3$:<br>
$x ^ 2 - 2x - 1$ 라는 2차 다항식에 $x=3$ 을 대입한 값을 계산하도록 하려면:



In [ ]:
second_order_polynomial(3, 1, -2, -1)



To calculate $x^2 - 4x + 4$ when $x=3$, we can update the coefficients as follows.<br>
$x^2 - 4x + 4$ 라는 2차 다항식에 $x=3$ 을 대입한 값을 계산하도록 하려면 다음과 같이 계수만 바꾸는 것으로 가능함.


In [ ]:
second_order_polynomial(3, 1, -4, 4)



#### x0 : `ndarray`



The second argument is the initial value of `x`. It would be advantageous to give a *sufficiently* close value. We may pass a float or `numpy.ndarray`.<br>
두번째 매개변수는 최적화를 시작할 초기값.  *충분히* 가까운 값을 주는 것이 유리.  `numpy` 의 (다차원) 배열 이라고 적혀 있으나 실수 float 값을 전달하는 것도 가능.



However, may not be that easy to find "how close"<br>
그런데, 얼마나 가까와야 하는가? 는 생각 보다 알기 힘들 수도 있음.



In [ ]:
def a_func_not_smooth(x):
    return np.sin(x*10) + 2.0 * x * x



In [ ]:
def plot_the_func_not_smooth():
    x_array = np.linspace(-1, 1, 200)
    y_array = a_func_not_smooth(x_array)
    plt.plot(x_array, y_array)
    plt.grid(True)


plot_the_func_not_smooth()
plt.show()
plt.close()



When "not close enough"<br>
"충분히 가깝지 않은" 경우



In [ ]:
not_the_best_initial_guess = -0.7
result_not_the_best_initial_guess = so.minimize(
    a_func_not_smooth,
    not_the_best_initial_guess,
    method="Nelder-Mead",
)

plot_the_func_not_smooth()
plt.plot(
    result_not_the_best_initial_guess.x,
    a_func_not_smooth(result_not_the_best_initial_guess.x),
    'o', label='final'
)
plt.plot(
    not_the_best_initial_guess,
    a_func_not_smooth(not_the_best_initial_guess),
    'x', label='initial'
)
plt.legend(loc=0)
plt.show()
plt.close()



**Luckily** "close enough"<br>
**운** 좋게 "충분히 가까운" 경우



In [ ]:
a_better_initial_guess = -0.3
result_a_better_initial_guess = so.minimize(
    a_func_not_smooth,
    a_better_initial_guess,
    method="Nelder-Mead"
)

plot_the_func_not_smooth()
plt.plot(
  result_a_better_initial_guess.x,
  a_func_not_smooth(result_a_better_initial_guess.x),
  'o', label='final'
)
plt.plot(
  a_better_initial_guess,
  a_func_not_smooth(a_better_initial_guess),
  'x', label='initial'
)
plt.legend(loc=0)
plt.show()
plt.close()



#### `args` : `tuple`, optional



Optional arguments of the function of the first argument. It is a fixed `tuple`.<br>
첫번째 매개변수로 전달한 함수의 두번째 이후 매개변수.  필요에 따라 사용. 튜플 `tuple` 이므로 내용이 변화하지 않음.



 For example, we can think about a second order polynomial as follows.<br>
 예를 들어 다음과 같이 여러 2차 다항식을 적용해 볼 수 있음.




In [ ]:
def second_order_polynomial(x, *args):
    a, b, c = args
    cost = a * x * x + b * x + c
    return cost



$$
f(x) = x^2 -2x -1
$$


In [ ]:
first_case = so.minimize(
    second_order_polynomial,
    0,
    args=(1, -2, -1),
    method="Nelder-Mead",
)
first_case



$$
f(x) = x^2 -4x+4
$$


In [ ]:
second_case = so.minimize(
    second_order_polynomial,
    0,
    (1, -4, +4),
    method="Nelder-Mead",
)
second_case



#### `method` : `str` or `callable`, optional



Choose minimization algorithm.<br>
최소화 알고리듬을 선택한다.

If unspecified, may use an algorithm such as `BFGS`.<br>
따로 선택하지 않으면 `BFGS` 등의 알고리듬을 사용한다.

The `BFGS` algorithm uses the first derivative of the cost function.<br>
해당 알고리듬은 비용함수의 1계 미분을 이용한다.

See argument `jac` for more about the first derivative of the cost function.<br>
비용함수의 1계 미분에 대해서는 `jac` 매개변수를 참고.

`Nedler-Mead` algorithm would minimize a function using the Simplex algorithm.<br>
`Nedler-Mead` 알고리듬의 경우, 어떤 함수를 심플렉스 알고리듬을 이용하여 최소화한다.

This algorithm only uses function values, not derivatives or second derivatives.<br>
이 알고리듬은 함수의 값만을 사용하며, 미분값이나 두번 미분한 값은 쓰지 않는다.



#### `jac` : callable `jac(x,*args)`, `"2-point"`, `"3-point"`, `"cs"`, `bool`, optional



Specifies how to calculate the gradient vector (or Jacobian).<br>경사 벡터를 (또는 쟈코비안을) 계산하는 법을 지정한다.

Only for the algorithms using the first derivatives.<br>1계 미분을 이용하는 알고리듬들만 해당된다.



## Summary<br>요약
Here we introduced possibly widely applicable `scipy.optimize.minimize()`. Please use responsibly.<br>여기서는 다양하게 활용할 수 있는 `scipy.optimize.minimize()`를 소개하였음. 책임감 있게 사용하기 바람.



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

